# **Finding Lane Lines on the Road** 

---

The goals / steps of this project are the following:
* Make a pipeline that finds lane lines on the road
* Reflect on your work in a written report


[//]: # (Image References)

[image1]: ./examples/grayscale.jpg "Grayscale"

---

### Reflection

### 1. Describe your pipeline. As part of the description, explain how you modified the draw_lines() function.

My pipeline consisted of 5 steps.
First,I converte the images to grayscale.
``` javascript 
    gray = grayscale(image)
```
Then I use gauss blur to reduce the noise point.
At first,the kernel size I’v been used was 3,but there is still have some noise point so I set to 5,and it is better. 
``` javascript 
    kernel_size = 5
    blur_gray = gaussian_blur(gray,kernel_size)
```
The next step is to detect edge by canny.
``` javascript
    low_threshold = 50
    high_threshold = 150
    img_canny = canny(blur_gray,low_threshold,high_threshold)
```
The next step is to get the region of interest that I use function as blew.
``` javascript 
    vertices = np.array([[(imshape[1]*0.15,imshape[0]*0.95),(imshape[1]*0.45, imshape[0]*0.59), (imshape[1]*0.55, imshape[0]*0.59), (imshape[1]*0.9,imshape[0]*0.95)]], dtype=np.int32)
    color_region = region_of_interest(img_canny,[vertices])
```
Finally I use houg_lines function to find lines,and combine two images together.
``` javascript
    #hough lines drawn to detect lines of a edge image
    rho = 3
    theta = np.pi/180
    threshold = 15
    min_line_length = 5
    max_line_gap = 25

    img_hough = hough_lines(color_region, rho, theta, threshold, min_line_length, max_line_gap)
    
    #combine hough lines and initial image
    img_weighted = weighted_img(img_hough,image)
```

In order to draw a single line on the left and right lanes, I modify the draw_lines() function by averaging the position of each of the lines and extrapolating to the top and bottom of the lane. 
Firstly,separating line segments by their slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left line vs. the right line. Based on the observation,the slope of lines in left part are larger than 0.5,while the slope of lines in right part are less than -0.5.
``` javascript
    if abs(x1-x2) == 0:
        slope = float("inf")
    else:
        slope = (y2 - y1)/(x2 - x1)

    if slope > 0.5 and x1 > x_size/2 and x2 > x_size/2:
        x_right.append(x1)
        x_right.append(x2)
        y_right.append(y1)
        y_right.append(y2)
    elif slope < -0.5 and x1 < x_size/2 and x2 < x_size/2:
        x_left.append(x1)
        x_left.append(x2)
        y_left.append(y1)
        y_left.append(y2)
``` 
Then fit a single line. We get slope and bias by using numpy.polyfit.
``` javascript
    slope, bias = np.polyfit(x_right, y_right, 1)
``` 
The next step is to average slope and bias.
``` javascript
    r_m = np.mean(x_right_all)
    r_b = np.mean(y_right_all)
``` 
Finally,we get top x1 and bottom x2 by averaged slope and bias.(top y1 and bottom y2 are preset)
``` javascript
    r_x1 = (y1 - r_b) / r_m
    r_x2 = (y2 - r_b) / r_m
```


### 2. Identify potential shortcomings with your current pipeline


There are many shortcoming in the curruet pipeline.

First of all,it can not work well with the challenge.I think there are many noise points in the challenge.

Another shortcoming could be adaptive. The test videos are special. The algorithm may fail on many other situations. For instance, weather and terrain.

At last,parameters are fixed,which may have great limitations. It takes a lot of time to improve the parameters in each situation.


### 3. Suggest possible improvements to your pipeline

A possible improvement would be to use color_select. In fact, in addition to white and yellow, road signs have a lot of color. It is helpfull to reduce noise points.

Another potential improvement could be to use a curve instead of a straight line. On a S-Bend road,straight line can not work well. lane line shoud be adaptive,no matter it's a straight line or a curve.
